In [24]:
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords

all_data = get_data_from_db()

num_remover = NumRemover()
word_remover = WordRemover(stopwords.words('portuguese'))
all_data = num_remover.fit_transform(all_data)
all_data = word_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
neu_data = all_data[all_data["labels"] == "NE"]
pos_data = all_data[all_data["labels"] == "PO"]

ratio = 0.4
neg_train = neg_data.iloc[0:round(ratio*neg_data.shape[0]), :]
neu_train = neu_data.iloc[0:round(ratio*neu_data.shape[0]), :]
pos_train = pos_data.iloc[0:round(ratio*pos_data.shape[0]), :]

neg_test = neg_data.iloc[round(ratio*neg_data.shape[0]): , :]
neu_test = neu_data.iloc[round(ratio*neu_data.shape[0]): , :]
pos_test = pos_data.iloc[round(ratio*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [16]:
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

vocab_neg = set(' '.join(neg_train["texts"].tolist()).split())
vocab_neu = set(' '.join(neu_train["texts"].tolist()).split())
vocab_pos = set(' '.join(pos_train["texts"].tolist()).split())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario neutro: " + str(len(vocab_neu)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 3485
Dimensão vocabulario neutro: 2759
Dimensão vocabulario positivo: 2973


### Removendo somente a interseção central

In [17]:
# Verificar interseção dos vocabulários
intersect = vocab_neg.intersection(vocab_pos.intersection(vocab_neu))
vocab_neg_excl = vocab_neg.difference(intersect)
vocab_neu_excl = vocab_neu.difference(intersect)
vocab_pos_excl = vocab_pos.difference(intersect)

lexicon = list(vocab_neg.union(vocab_pos))

In [18]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon, binary = True))
                    ])

folds = 10;
print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds= folds)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=folds)

print("\nMaxent")
run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds=folds)

MultinomialNB
Cross Validation:
Accuracia media:  0.541346153846
Desvio padrão:  0.0391878536006


SVM
Cross Validation:
Accuracia media:  0.552703193033
Desvio padrão:  0.0469909981568

Maxent
Cross Validation:
Accuracia media:  0.570881712627
Desvio padrão:  0.036761742242


In [19]:
def get_ngram_lexicon(ngram_range):
    # Avaliando a incidência dos bigramas
    cv = CountVectorizer(ngram_range=ngram_range, stop_words = stopwords_pt)

    neg_counts = cv.fit_transform(neg_train["texts"])
    neg_ngrams = set(cv.vocabulary_)

    pos_counts = cv.fit_transform(pos_train["texts"])
    pos_ngrams = set(cv.vocabulary_)

    neu_counts = cv.fit_transform(neu_train["texts"])
    neu_ngrams = set(cv.vocabulary_)

    # Encontrando as interseções
    central_intersection = pos_ngrams.intersection(neg_ngrams).intersection(neu_ngrams)
    pos_neg_intersection = pos_ngrams.intersection(neg_ngrams)
    pos_neu_intersection = pos_ngrams.intersection(neu_ngrams)
    neu_neg_intersection = neu_ngrams.intersection(neg_ngrams)

    total_intersection = central_intersection.union(pos_neg_intersection).union(pos_neu_intersection).union(neu_neg_intersection)

    #Removendo as interseções do vocabulario
    lexicon_ngrams = neg_ngrams.union(pos_ngrams).union(neu_ngrams).difference(total_intersection)

    print("Dimensionalidade: ", len(lexicon_ngrams))
    return lexicon_ngrams

# neg_counts = cv.fit_transform(neg_train["texts"])
# neg_ngrams = cv.vocabulary_

# pos_counts = cv.fit_transform(pos_train["texts"])
# pos_ngrams = cv.vocabulary_

# neu_counts = cv.fit_transform(neu_train["texts"])
# neu_ngrams = cv.vocabulary_

# spo = np.sum(pos_counts.toarray(), axis=0)
# sng = np.sum(neg_counts.toarray(), axis=0)
# sne = np.sum(neu_counts.toarray(), axis=0)

# pos_ngrams  = {ngram: spo[index] for ngram, index in pos_ngrams.items()}
# neg_ngrams  = {ngram: sng[index] for ngram, index in neg_ngrams.items()}
# neu_ngrams  = {ngram: sne[index] for ngram, index in neu_ngrams.items()}

# pos_excl_ngrams = {key:value for key, value in pos_ngrams.items() if key not in neg_ngrams and key not in neu_ngrams}
# neg_excl_ngrams = {key:value for key, value in neg_ngrams.items() if key not in pos_ngrams and key not in neu_ngrams}
# neu_excl_ngrams = {key:value for key, value in neu_ngrams.items() if key not in pos_ngrams and key not in neg_ngrams}

# lexicon_ngrams = list(pos_excl_ngrams.keys()) + list(neg_excl_ngrams.keys()) + list(neu_excl_ngrams)

In [4]:
from pandas import Series
s = Series(neg_excl_ngrams)
s.nlargest(15)

NameError: name 'neg_excl_ngrams' is not defined

### Somente com Unigrams

In [20]:
lexicon_ngrams = get_ngram_lexicon(ngram_range=(1,1))

print("UNIGRAMS FREQUENCY**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,1), stop_words=stopwords_pt, vocabulary= lexicon_ngrams))
                    ])
evaluate(all_data, features, n_folds= 10)

print("\n\n")
print("UNIGRAMS PRESENCE**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,1), stop_words=stopwords_pt, vocabulary= lexicon_ngrams, binary= True))
                    ])
evaluate(all_data, features, n_folds= 10)

Dimensionalidade:  3454
UNIGRAMS FREQUENCY**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.687137155298
Desvio padrão:  0.0562521203575

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.71692670537
Desvio padrão:  0.0542063178274

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.620917997097
Desvio padrão:  0.0526882649495



UNIGRAMS PRESENCE**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.686175616836
Desvio padrão:  0.0482760615473

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.71692670537
Desvio padrão:  0.0521193956343

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.627648766328
Desvio padrão:  0.056747171455


In [21]:
len(lexicon_ngrams)

3454

### Somente com Bigrams

In [22]:
lexicon_ngrams = get_ngram_lexicon(ngram_range=(2,2))
print("\n Dimensionalidade: ", len(lexicon_ngrams))

print("BIGRAMS FREQUENCY**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(2,2), stop_words=stopwords_pt, vocabulary= lexicon_ngrams))
                    ])
evaluate(all_data, features, n_folds= 10)

print("\n\n")

print("BIGRAMS PRESENCE**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(2,2), stop_words=stopwords_pt, vocabulary= lexicon_ngrams, binary= True))
                    ])
evaluate(all_data, features, n_folds= 10)

Dimensionalidade:  17501

 Dimensionalidade:  17501
BIGRAMS FREQUENCY**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.616092162554
Desvio padrão:  0.0412408122012

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.73898766328
Desvio padrão:  0.0393995760161

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.443468795356
Desvio padrão:  0.0259476079279



BIGRAMS PRESENCE**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.616092162554
Desvio padrão:  0.043211395596

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.73898766328
Desvio padrão:  0.0393995760161

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.440602322206
Desvio padrão:  0.026336277332


### Bigrams + Unigrams

In [25]:
lexicon_ngrams = get_ngram_lexicon(ngram_range=(1,2))

print("UNIGRAMS + BIGRAMS FREQUENCY**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_ngrams))
                    ])
evaluate(all_data, features, n_folds= 10)

print("\n\n")

print("UNIGRAMS + BIGRAMS PRESENCE**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_ngrams, binary= True))
                    ])
evaluate(all_data, features, n_folds= 10)

Dimensionalidade:  9980
UNIGRAMS + BIGRAMS FREQUENCY**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.589277939042
Desvio padrão:  0.0483179495124

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.57578011611
Desvio padrão:  0.0496045369628

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.522115384615
Desvio padrão:  0.0354728206648



UNIGRAMS + BIGRAMS PRESENCE**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.5912191582
Desvio padrão:  0.0481898747099

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.589169085631
Desvio padrão:  0.055376583559

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.525925253991
Desvio padrão:  0.0359972714007


### Usando LSA e MaxEnt

In [12]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion

pca = PCA()
# pca = TruncatedSVD(n_components=500)
cv  = CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_ngrams)

t_data = cv.fit_transform(all_data["texts"])
t_data = pca.fit_transform(t_data.toarray())

labels = np.array([])
for label in all_data["labels"]:
    if label == 'PO':
        labels = np.append(labels, [1])
    elif label == "NG":
        labels = np.append(labels, [-1])
    elif label == "NE":
        labels = np.append(labels, [0])

labels.shape

classifier = LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False)
# classifier = MultinomialNB()

print("Cross Validation:")
accuracy_average = np.array([])
sKFold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= True)
for index, (train, test) in enumerate(sKFold.split(t_data, labels)):
    # Treinando um modelo Naive Bayes
    train_data = t_data[train]
    test_data = t_data[test]

    classifier.fit(train_data, labels[train])
    predictions = classifier.predict(test_data)

    accuracy = accuracy_score(labels[test], predictions, normalize=True)
    # classifier_models.append(pipeline)

    accuracy_average = np.append(accuracy_average, accuracy)
    #print("Fold ", index, " - Acuracia: ", accuracy)

print("Accuracia media: ", accuracy_average.mean())
print("Desvio padrão: ", accuracy_average.std())

Cross Validation:
Accuracia media:  0.416509433962
Desvio padrão:  0.0490313027867
